In [ ]:
%matplotlib inline
import ipympl
import sys
sys.path.insert(1, 'functions')
import numpy as np
import pandas as pd
import scipy.io
import scipy
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import preprocessing
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import normaltest
from normality_function import normality_fun
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
import statsmodels
from scipy.interpolate import interp1d
from scipy import signal
from scipy.fft import fftshift
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics
import statsmodels.tsa
from matplotlib import cm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics
from scipy.interpolate import interp1d
import math
%matplotlib widget

plt.ion()
# Import mat files
mat_angle_40bpm = scipy.io.loadmat('data/angle_40bpm.mat')
mat_emg1_40bpm = scipy.io.loadmat('data/emg1_40bpm.mat')
mat_emg2_40bpm = scipy.io.loadmat('data/emg2_40bpm.mat')

# Convert them to numpy arrays
angle = np.array(mat_angle_40bpm['angle1'])
emg1 = np.array(mat_emg1_40bpm['emg1'])
emg2 = np.array(mat_emg2_40bpm['emg2'])
T = 0.001
N = emg1.shape[0]

# Normalize data
scaler = preprocessing.StandardScaler().fit(emg1)
emg1 = scaler.transform(emg1)
scaler = preprocessing.StandardScaler().fit(emg2)
emg2 = scaler.transform(emg2)
scaler = preprocessing.StandardScaler().fit(angle)
angle = scaler.transform(angle)

# Create Interpolated version of angles
old_indices = np.arange(0, angle.shape[0])
new_length = emg1.shape[0]
new_indices = np.linspace(0, angle.shape[0] - 1, new_length)
inter = interp1d(old_indices, angle, axis=0)
angle = np.array(inter(new_indices))

In [ ]:
tot_length = 5000
wind = np.ones([50])/50
fig, axs = plt.subplots(2,1)
axs[0].set_xlabel("frequency (Hz)", fontsize=20)
axs[0].set_ylabel("PSD (dB) ", fontsize=20)
axs[1].set_xlabel("frequency (Hz)", fontsize=20)
axs[1].set_ylabel("PSD MA(50) (dB) ", fontsize=20)
fig.suptitle(" Comparative 14-segment Welch for biceps and triceps signal and MA-50", fontsize=22)
for i in range(14):
    f, Pxx = scipy.signal.welch(emg1[i*tot_length:(i+1)*tot_length].reshape(-1), fs=1000, window='hann', nperseg=2000, noverlap=int(0.85*2000))
    axs[0].semilogy(f, Pxx, label=f"Segment {i+1}")
    Pxx = scipy.signal.convolve(Pxx.reshape(-1), wind, mode='same')
    axs[1].semilogy(f, Pxx, label=f"Segment {i+1} MA(50)")
legend = axs[0].legend(loc='upper center', shadow=True, fontsize='x-large')
axs[0].legend(loc='upper right')
axs[1].legend(loc='upper right')
plt.show()

fig, axs = plt.subplots(2,1)
axs[0].set_xlabel("frequency (Hz)", fontsize=20)
axs[0].set_ylabel("PSD (dB) ", fontsize=20)
axs[1].set_xlabel("frequency (Hz)", fontsize=20)
axs[1].set_ylabel("PSD MA(50) (dB) ", fontsize=20)
fig.suptitle(" Comparative 14-segment Welch for triceps signal and MA-50", fontsize=22)
for i in range(14):
    f, Pxx = scipy.signal.welch(emg2[i*tot_length:(i+1)*tot_length].reshape(-1), fs=1000, window='hann', nperseg=2000, noverlap=int(0.85*2000))
    axs[0].semilogy(f, Pxx, label=f"Segment {i+1}")
    Pxx = scipy.signal.convolve(Pxx.reshape(-1), wind, mode='same')
    axs[1].semilogy(f, Pxx, label=f"Segment {i+1} MA(50)")
    
axs[0].legend(loc='upper right')
axs[1].legend(loc='upper right')
plt.show()